----
----
-----

## Group Normalization’da Mean ve Std Ne Oluyor?

Group Normalization (GN) uygulandığında hedeflenen durum şudur:

- **Ortalama (mean) = 0**
- **Standart sapma (std) = 1**

Ancak burada kritik olan nokta **neyin** 0–1 yapıldığıdır.


## GN’de Neyi 0–1 Yapıyoruz?

GN, **her örnek (sample)** için ve **her grup özelinde** çalışır.

İzlenen adımlar:

- Kanallar (`C`) belirli sayıda **gruba ayrılır**
- Her grup için, o gruba ait:
  - kanallar
  - uzaysal boyutlar (`H, W`)
  birlikte ele alınır
- Bu grup içindeki aktivasyonlar için:
  - **mean → 0**
  - **std → 1**
  olacak şekilde normalize edilir

Yani GN:

- ❌ Tüm **batch** üzerinde normalize etmez  
- ❌ Tüm **kanalları tek seferde** normalize etmez  
- ✅ **Grup bazlı** normalize eder


## Neden Ölçekliyoruz? (Asıl Amaç)

### 1️⃣ Optimizasyonu Stabilize Etmek
- Aktivasyonlar çok büyürse → **gradyan patlaması**
- Aktivasyonlar çok küçülürse → **gradyan sönmesi**
- GN, her grubun aktivasyon ölçeğini **kontrol altında tutar**


### 2️⃣ Batch’e Bağımlılığı Kırmak
- Batch Normalization, batch istatistiklerine bağımlıdır
- GN, **batch size’dan bağımsızdır**
- Batch küçük olsa bile (1–2–4):
  - eğitim **stabil** kalır


### 3️⃣ Kanal Gruplarını Dengede Tutmak
- CNN’lerde bazı kanal grupları baskın hâle gelebilir
- GN, her gruba **benzer ölçek** kazandırır
- Böylece:
  - feature learning daha **dengeli**
  - temsil gücü daha **istikrarlı** olur


## Kısa Özet

> Group Normalization’da da hedeflenen durum **mean=0, std=1**’dir.  
> Ancak bu, batch genelinde değil; **her örnek ve her grup özelinde** yapılır.  
> Amaç bir loss minimize etmek değil, CNN eğitimini **small-batch rejiminde stabil hâle getirmektir.**

----
----
----

# 4) Normalization Katmanları — Group Normalization (GroupNorm)

Bu defter, **Group Normalization (GN)** kavramını **sıfırdan ileri seviyeye** taşır:
- GN nedir, neden ortaya çıkmıştır?
- Matematiksel tanım (forward) ve eksen mantığı
- CNN (Object Detection / Segmentation) tarafında neden çok popüler?
- PyTorch kullanımı: `nn.GroupNorm`
- `num_groups` seçimi için pratik kurallar
- BN/LN/IN ile karşılaştırma + küçük batch demo

> Not: Odak **CNN pratiği**dir. Gereksiz teori yerine: doğru sezgi + doğru kullanım + doğru kod.


---

## 0) GN’yi anlamak için 1 cümle

**GroupNorm**, her örnek (sample) için, **kanalları (C) gruplara bölüp** her grubu ayrı normalize eder.

- BN: batch üzerinden istatistik (batch'e bağımlı)
- IN: her kanal ayrı normalize (çok agresif)
- LN: tüm kanallar birlikte normalize (tek grup)
- **GN: kanallar gruplara ayrılır (orta yol)** ✅


---

## 1) GN nereden çıkmıştır? Kullanılma amacı nedir?

CNN tabanlı **object detection** ve **segmentation** eğitiminde batch genelde küçüktür:
- Yüksek çözünürlük (H,W büyük)
- Büyük backbone/neck/head
- VRAM limiti → batch=1/2/4

Bu durumda **BatchNorm** istatistikleri (mean/var) gürültülü olur.
**GroupNorm**, batch'e ihtiyaç duymadan (batch size'dan bağımsız) stabil eğitim sağlar.

Özet: GN’nin ana amacı:
- **Small-batch rejiminde** BN’nin zayıflığını kapatmak
- Train/infer davranışını basitleştirmek (BN’deki running stats karmaşası yok)


---

## 2) GN neyi normalize eder? (eksensel bakış)

Giriş tensorü: `x ∈ R^{N×C×H×W}` (NCHW)

GN:
1. `C` kanalı **G gruba** böler.
2. Her grup için, ilgili kanalların tüm uzaysal boyutlarıyla birlikte (H,W) istatistik hesaplar.
3. Normalize eder, sonra affine (`γ,β`) uygular.

Yani normalize edilen set:
- **(C/G) kanalı + H + W** (her örnek için, grup başına)

Bu yüzden GN:
- Batch’ten bağımsız ✅
- CNN’de NCHW formatıyla doğal uyumlu ✅


### 2.1) Matematik (forward)

Bir örnek için, grup indeksi `g` olsun.

- O gruba ait kanal kümesi `S_g` (|S_g| = C/G)
- Normalize edilen eleman sayısı: `m = (C/G) * H * W`

Ortalama:
\[
\mu_g = \frac{1}{m}\sum_{i \in S_g,\,h,w} x_{i,h,w}
\]

Varyans:
\[
\sigma_g^2 = \frac{1}{m}\sum_{i \in S_g,\,h,w} (x_{i,h,w}-\mu_g)^2
\]

Normalize:
\[
\hat{x} = \frac{x - \mu_g}{\sqrt{\sigma_g^2+\epsilon}}
\]

Affine:
\[
y = \gamma \hat{x} + \beta
\]

Not:
- `γ, β` kanal bazlıdır (C boyutunda).


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)

---

## 3) PyTorch’ta GN: `nn.GroupNorm`

Kullanım:
```python
nn.GroupNorm(num_groups=G, num_channels=C, eps=1e-5, affine=True)
```

- `num_groups` = kaç grup
- `num_channels` = kanal sayısı
- GN, NCHW ile direkt çalışır ✅
- BN gibi running stats yok → train/infer aynı mantık ✅


In [ ]:
x = torch.randn(2, 8, 4, 4)  # (N=2, C=8, H=4, W=4)

gn = nn.GroupNorm(num_groups=4, num_channels=8)  # 8 kanal -> 4 grup -> grup başına 2 kanal
y = gn(x)

print("x shape:", x.shape)
print("y shape:", y.shape)

x shape: torch.Size([2, 8, 4, 4])
y shape: torch.Size([2, 8, 4, 4])


---

## 4) GN’nin “LN/IN” ile ilişkisi (çok önemli!)

GN aslında bir aile gibi düşünülebilir:

- **GN(G=C)** → her kanal ayrı normalize ⇒ **InstanceNorm benzeri**
- **GN(G=1)** → tüm kanallar tek grup ⇒ **LayerNorm benzeri**
- **GN(ara G)** → ikisinin arası ⇒ **GroupNorm**

Bu, GN’nin neden “esnek” olduğunu açıklar.


In [ ]:
C = 8
x = torch.randn(2, C, 4, 4)

gn_like_ln = nn.GroupNorm(num_groups=1, num_channels=C)   # LN benzeri
gn_like_in = nn.GroupNorm(num_groups=C, num_channels=C)   # IN benzeri

y_ln = gn_like_ln(x)
y_in = gn_like_in(x)

print("GN(1) output mean (sample0):", y_ln[0].mean().item())
print("GN(C) output mean (sample0):", y_in[0].mean().item())

GN(1) output mean (sample0): 1.862645149230957e-09
GN(C) output mean (sample0): 2.2351741790771484e-08


---

## 5) `num_groups` nasıl seçilir? (pratik kurallar)

Genel öneriler (CNN/detection/segmentation):
- **32 grup** yaygın default
- Ama `C`, `G`’ye bölünebilmelidir: `C % G == 0`

Pratik kural seti:

1) Önce `G=32` dene  
2) Eğer `C < 32` ise `G=16 / 8 / 4` dene  
3) Eğer `C` çok küçükse `G=1` (LN benzeri) veya `G=C` (IN benzeri)

Aşağıdaki fonksiyon “makul GN” seçimi yapar.


In [ ]:
def choose_gn_groups(C: int, preferred=(32, 16, 8, 4, 2, 1)):
    for g in preferred:
        if C % g == 0:
            return g
    return 1

for C in [24, 32, 48, 64, 96, 128, 160, 192, 256]:
    print(C, "-> groups:", choose_gn_groups(C))

24 -> groups: 8
32 -> groups: 32
48 -> groups: 16
64 -> groups: 32
96 -> groups: 32
128 -> groups: 32
160 -> groups: 32
192 -> groups: 32
256 -> groups: 32


---

## 6) GN vs BN: CNN pratik farkları

### Batch bağımlılığı
- BN: batch istatistikleri → batch küçükse gürültü
- GN: örnek içi grup istatistiği → batch’ten bağımsız ✅

### Train/Inference
- BN: train ve inference farklı (running mean/var)
- GN: aynı davranır ✅

### Performans (genel gözlem)
- Büyük batch classification: BN genelde daha iyi/hızlı
- Small batch detection/segmentation: GN genelde daha stabil ✅


---

## 7) Mini deney: küçük batch'te BN vs GN (hızlı demo)

Bu demo:
- Aynı küçük CNN’i BN ile ve GN ile kurar
- Batch size=2 ve 16 için 1 epoch loss kıyaslar
- Benchmark değildir; sezgi içindir.


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor()])
train_ds = datasets.FakeData(size=512, image_size=(3, 32, 32), num_classes=10, transform=transform)

def make_loader(batch_size):
    return DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)

class SmallCNN_BN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
        )
        self.fc = nn.Linear(64, 10)
    def forward(self, x):
        x = self.net(x).flatten(1)
        return self.fc(x)

class SmallCNN_GN(nn.Module):
    def __init__(self, g1=8, g2=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.GroupNorm(g1, 32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.GroupNorm(g2, 64),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
        )
        self.fc = nn.Linear(64, 10)
    def forward(self, x):
        x = self.net(x).flatten(1)
        return self.fc(x)

def one_epoch(model, loader, lr=1e-2, device="cpu"):
    model.to(device)
    model.train()
    opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    total_loss = 0.0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        opt.zero_grad(set_to_none=True)
        logits = model(x)
        loss = F.cross_entropy(logits, y)
        loss.backward()
        opt.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(loader.dataset)

for bs in [2, 16]:
    loader = make_loader(bs)
    bn_model = SmallCNN_BN()
    gn_model = SmallCNN_GN(g1=8, g2=16)
    bn_loss = one_epoch(bn_model, loader)
    gn_loss = one_epoch(gn_model, loader)
    print(f"Batch size={bs:>2} | BN loss={bn_loss:.4f} | GN loss={gn_loss:.4f}")


Batch size= 2 | BN loss=2.4127 | GN loss=2.4428
Batch size=16 | BN loss=2.3237 | GN loss=2.3522


---

## 8) Detection/Segmentation için GN nerede kullanılır?

- **Backbone**: BN yerine GN (small batch)
- **Neck (FPN/PAN)**: ölçekler arası karışımda stabilite
- **Head**: özellikle segmentation head’lerinde çok yaygın

Kısa not:
- Multi-GPU ve efektif batch büyükse: SyncBN iyi alternatif
- Tek GPU / küçük batch: GN çoğu zaman daha az problem çıkarır ✅


---

## 9) Weight Standardization (WS) ile ilişkisi (kısa)

GN ile birlikte bazen **Weight Standardization** kullanılır:
- GN: aktivasyonu normalize eder
- WS: ağırlıkları normalize eder

Bu kombinasyon, küçük batch rejiminde BN benzeri stabiliteyi yakalamaya yardımcı olabilir.
(GN oturduktan sonra WS daha anlamlı gelir.)


---

## 10) Özet

- **GroupNorm**, kanalları gruplara bölerek normalize eder.
- Batch bağımsızdır → **small-batch detection/segmentation** için çok uygundur.
- `num_groups` seçimi önemlidir (genelde 32/16/8).
- GN ailesi:
  - GN(1) ≈ LN benzeri
  - GN(C) ≈ IN benzeri
- PyTorch’ta doğrudan `nn.GroupNorm` ile NCHW üzerinde kullanılır.

Bir sonraki defter: **InstanceNorm** — ne zaman işe yarar, CNN’de nerede riskli olabilir?
